In [3]:
import zipfile
zip_file_path = '/content/lung_cancer_MRI_dataset.zip'
extract_dir = '/content/lung_cancer_MRI_dataset'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
    print("Extraction completed.")

Extraction completed.


In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_dir = '/content/lung_cancer_MRI_dataset/lung_cancer_MRI_dataset/train'
val_dir = '/content/lung_cancer_MRI_dataset/lung_cancer_MRI_dataset/validate'
img_size = (224, 224)
batch_size = 32
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model.trainable=False
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions= Dense(train_generator.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
epochs = 10
history = model.fit(train_generator, steps_per_epoch=train_generator.samples // batch_size, validation_data=val_generator, validation_steps=val_generator.samples // batch_size, epochs=epochs)
loss, accuracy = model.evaluate(val_generator)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")
model.save('my_model.keras')

Found 2436 images belonging to 2 classes.
Found 1244 images belonging to 2 classes.


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling_2 (Rescaling)   │ (None, 224, 224, 3)    │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ normalization_1           │ (None, 224, 224, 3)    │              7 │ rescaling_2[0][0]      │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling_3 (Rescaling)   │ (None, 224, 224, 3)    │              0 │ normalization_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv_pad             │ (None, 225, 225, 3)    │              0 │ rescaling_3[0][0]      │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv (Conv2D)        │ (None, 112, 112, 32)   │            864 │ stem_conv_pad[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_bn                   │ (None, 112, 112, 32)   │            128 │ stem_conv[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_activation           │ (None, 112, 112, 32)   │              0 │ stem_bn[0][0]          │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_dwconv            │ (None, 112, 112, 32)   │            288 │ stem_activation[0][0]  │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_bn                │ (None, 112, 112, 32)   │            128 │ block1a_dwconv[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_activation        │ (None, 112, 112, 32)   │              0 │ block1a_bn[0][0]       │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_squeeze        │ (None, 32)             │              0 │ block1a_activation[0]… │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reshape        │ (None, 1, 1, 32)       │              0 │ block1a_se_squeeze[0]… │
│ (Reshape)                 │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reduce         │ (None, 1, 1, 8)        │            264 │ block1a_se_reshape[0]… │
│ (Conv2D)                  │                        │                │                        │
├──────────────────────

 Total params: 5,363,365 (20.46 MB)

 Trainable params: 1,313,794 (5.01 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

Epoch 1/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 313s 4s/step - accuracy: 0.4904 - loss: 0.8599 - val_accuracy: 0.3947 - val_loss: 0.7315
Epoch 2/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 139s 2s/step - accuracy: 0.4375 - loss: 0.7182 - val_accuracy: 0.3956 - val_loss: 0.7440
Epoch 3/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 325s 4s/step - accuracy: 0.4933 - loss: 0.7059 - val_accuracy: 0.3964 - val_loss: 0.7405
Epoch 4/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 142s 2s/step - accuracy: 0.4375 - loss: 0.7306 - val_accuracy: 0.3964 - val_loss: 0.7289
Epoch 5/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 382s 5s/step - accuracy: 0.5472 - loss: 0.6920 - val_accuracy: 0.4005 - val_loss: 0.6963
Epoch 6/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 91s 1s/step - accuracy: 0.3750 - loss: 0.6944 - val_accuracy: 0.3956 - val_loss: 0.6966
Epoch 7/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 322s 4s/step - accuracy: 0.5155 - loss: 0.6950 - val_accuracy: 0.3964 - val_loss: 0.7398
Epoch 8/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 140s 2s/step - accuracy: 0.5625 - loss: 0.6883 - val_accuracy: 0.3964 - va